In [119]:
import pandas as pd
import numpy as np
train = pd.read_csv("train_data1.csv", encoding='cp949')
test = pd.read_csv('test_data.csv', encoding='cp949')

시간당 내부 평균온도: 4도~40도
시간당 내부 평균습도: 0% ~ 100%
시간당 내부 평균 co2 농도 : 0ppm ~ 1200 ppm
시간당 평균 EC : 0 ~ 8
시간당 분무량 : 0 ~ 3000 / 일간 누적 분무량 0 ~ 72,000
시간당 백색광량 : 0 ~ 120,000 / 일간 누적 백색광량 0 ~ 2,880,000
시간당 적색광량 : 0 ~ 120,000 / 일간 누적 적색광량 0 ~ 2,880,000 
시간당 청색광량 : 0 ~ 120,000 / 일간 누적 청색광량 0 ~ 2,880,000
시간당 총광량 : 0 ~ 120,000 / 일간 누적 총광량 0 ~ 2,880,000

In [120]:
def out_value(df):
    df.loc[(df['내부온도관측치'] < 4) | (df['내부온도관측치'] > 40), '내부온도관측치'] = np.nan
    df.loc[(df['내부습도관측치'] < 0) | (df['내부습도관측치'] > 100), '내부습도관측치'] = np.nan
    df.loc[(df['co2관측치'] < 0) | (df['co2관측치'] > 1200), 'co2관측치'] = np.nan
    df.loc[(df['ec관측치'] < 0) | (df['ec관측치'] > 8), 'ec관측치'] = np.nan
    df.loc[(df['시간당분무량'] < 0) | (df['시간당분무량'] > 3000), '시간당분무량'] = np.nan
    df.loc[(df['일간누적분무량'] < 0) | (df['일간누적분무량'] > 72000), '일간누적분무량'] = np.nan
    df.loc[(df['시간당백색광량'] < 0) | (df['시간당백색광량'] > 120000), '시간당백색광량'] = np.nan
    df.loc[(df['일간누적백색광량'] < 0) | (df['일간누적백색광량'] > 2880000), '일간누적백색광량'] = np.nan
    df.loc[(df['시간당적색광량'] < 0) | (df['시간당적색광량'] > 120000), '시간당적색광량'] = np.nan
    df.loc[(df['일간누적적색광량'] < 0) | (df['일간누적적색광량'] > 2880000), '일간누적적색광량'] = np.nan
    df.loc[(df['시간당청색광량'] < 0) | (df['시간당청색광량'] > 120000), '시간당청색광량'] = np.nan
    df.loc[(df['일간누적청색광량'] < 0) | (df['일간누적청색광량'] > 2880000), '일간누적청색광량'] = np.nan
    df.loc[(df['시간당총광량'] < 0) | (df['시간당총광량'] > 120000), '시간당총광량'] = np.nan
    df.loc[(df['일간누적총광량'] < 0) | (df['일간누적총광량'] > 2880000), '일간누적총광량'] = np.nan
    return df

In [121]:
train = out_value(train)
test = out_value(test)
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

In [122]:
train['obs_time'] = [i[:2] for i in train['obs_time']]
test['obs_time'] = [i[:2] for i in test['obs_time']]

In [127]:
val = train[:672]
train = train[672:]

In [129]:
x_train = train.drop(['predicted_weight_g', 'Case'], axis=1)
y_train = train['predicted_weight_g']

x_val = val.drop(['predicted_weight_g', 'Case'], axis=1)
y_val = val['predicted_weight_g']

x_test = test.drop(['predicted_weight_g', 'Case'], axis=1)

In [130]:
from catboost import CatBoostRegressor
model = CatBoostRegressor()
model.fit(x_train, y_train, verbose=100)

Learning rate set to 0.064723
0:	learn: 40.0126944	total: 4.43ms	remaining: 4.43s
100:	learn: 9.3426753	total: 420ms	remaining: 3.73s
200:	learn: 6.9547623	total: 770ms	remaining: 3.06s
300:	learn: 5.7424861	total: 1.11s	remaining: 2.58s
400:	learn: 5.0147213	total: 1.44s	remaining: 2.16s
500:	learn: 4.5164960	total: 1.77s	remaining: 1.77s
600:	learn: 4.1320910	total: 2.13s	remaining: 1.41s
700:	learn: 3.8208680	total: 2.46s	remaining: 1.05s
800:	learn: 3.5574591	total: 2.81s	remaining: 697ms
900:	learn: 3.3409143	total: 3.15s	remaining: 346ms
999:	learn: 3.1603357	total: 3.47s	remaining: 0us


In [136]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(y_val, y_pred)**0.5)

15.36582042764997


In [17]:
y_pred = model.predict(x_test)

In [20]:
x_test['predicted_weight_g'] = y_pred

In [24]:
submit = x_test[['DAT', 'predicted_weight_g']]

In [34]:
submit['DAT'] = submit['DAT']+1

C:\Users\kki96\AppData\Local\Temp\ipykernel_19540\2221344854.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['DAT'] = submit['DAT']+1


In [35]:
submit

,DAT,predicted_weight_g
0,1,1.003079
1,1,1.426150
2,1,0.970993
3,1,1.001610
4,1,1.090658
...,...,...
3355,28,95.442424
3356,28,96.520630
3357,28,92.706644
3358,28,89.205140


In [48]:
len(y_pred)

3360

In [49]:
submit

,DAT,predicted_weight_g
0,1,1.003079
1,1,1.426150
2,1,0.970993
3,1,1.001610
4,1,1.090658
...,...,...
3355,28,95.442424
3356,28,96.520630
3357,28,92.706644
3358,28,89.205140


In [ ]:
import glob
all_target_list = sorted(glob.glob('./data/test_target/*.csv'))


,DAT,predicted_weight_g
0,1,1.003079
1,1,1.426150
2,1,0.970993
3,1,1.001610
4,1,1.090658
...,...,...
3355,28,95.442424
3356,28,96.520630
3357,28,92.706644
3358,28,89.205140


In [62]:
test_Df = pd.read_csv('TEST_ALL.csv')

In [ ]:
for i in range(140):
    test_Df['predicted_weight_g'][i] = y_pred[i*24:i*24+24].mean()

In [118]:
import glob
all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
for idx, test_path in enumerate(all_target_list):
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = test_Df['predicted_weight_g'][idx*28:idx*28+28].values
    submit_df.to_csv(test_path, index=False)
    
    